# Dataset Collection and Analysis for MOBs

## 1. Issue Selecion

We use GitHub API to search the issues.

In [1]:
! pip install -q PyGithub

from github import Github

YOUR_GITHUB_TOKEN = ""
if len(YOUR_GITHUB_TOKEN) == 0:
    raise ValueError("Please set your GitHub token in the script")

g = Github(YOUR_GITHUB_TOKEN)

The query rules are described in Section 3.2:

In [2]:
queries = [
    ("pytorch/pytorch", "label:\"module: pruning\" label:triaged is:issue"),
    ("pytorch/pytorch", "label:\"oncall: quantization\" label:triaged is:issue"),
    ("tensorflow/tensorflow", "label:ModelOptimizationToolkit label:type:bug is:issue"),
    ("tensorflow/model-optimization", "label:bug is:issue"),
]

### Search and save raw data.

In [56]:
import json
full_data = []

for q in queries:
    issues = g.search_issues(query=f'repo:{q[0]} {q[1]}')
    
    for issue in issues:
        full_data.append(issue.raw_data)
    print(issues.totalCount)

15
321
32
127


In [59]:
with open('full_data_new.json', 'w') as f:
    json.dump(full_data, f)

In [29]:
valid_issues = []
for issue in full_data:
    if issue['comments'] == 0:
        continue
    body = issue['body']
    if 'Feature\r\n' in body:
        continue
    if 'Motivation\r\n' in body:
        continue
    if 'Documentation\r\n' in body:
        continue
    if 'The feature, motivation and pitch\r\n' in body:
        continue
    valid_issues.append(issue)

print(len(valid_issues))


393


In [ ]:
import csv
import json
import re

with open('full_data_new.json', 'r') as f:
    full_data = json.load(f)

print(len(full_data))

valid_issues = []
for issue in full_data:
    if issue['comments'] == 0:
        continue
    body = issue['body']
    if 'Feature\r\n' in body:
        continue
    if 'Motivation\r\n' in body:
        continue
    if 'Documentation\r\n' in body:
        continue
    if 'The feature, motivation and pitch\r\n' in body:
        continue
    title = issue['title']
    if 'test_' in title:
        continue
    valid_issues.append(issue)

print(len(valid_issues))

no_code_issues = []
for issue in valid_issues:
    body = issue['body']
    if '`' in body:
        continue
    if 'To Reproduce\r\n' in body:
        continue
    if 'https://' in body:
        continue
    if 'torch.' in body:
        continue
    if 'tensorflow.' in body:
        continue
    if len(re.findall(r'#\d+', body)) > 0:
        continue
    no_code_issues.append(issue)
    print(issue['html_url'])
print(len(no_code_issues))

In [ ]:
import re
import requests

ref_pattern = re.compile(r'#(\d+)')
client = requests.Session()
client.headers["Authorization"] = f"token {YOUR_GITHUB_TOKEN}"

for issue in valid_issues:
    r = client.get(issue['timeline_url'])
    for event in r.json():
        if event['event'] == 'cross-referenced':
            source = event['source']
            if source['type'] == 'issue':
                if 'pull_request' in source['issue']:
                    pr_url = source['issue']['pull_request']['url']
                    pr = client.get(pr_url).json()
                    if pr['merged_at'] is not None:
                        print(issue['html_url'], pr['html_url'])
                    for label in pr['labels']:
                        if label['name'] == 'Merged':
                            print(issue['html_url'], pr['html_url'])
                            break
